In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import model_evaluation_functions

In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Load and clean prediciton data

In [4]:
df_ground_truth = pd.read_csv('../annotations/ground_truth_boat_frames_coco.csv', index_col=0, sep=';')
df_yolo_predictions = pd.read_csv('../annotations/yolov8x_single_camera_data_train_video_labels.csv', index_col=0, sep=';')

df_ground_truth['datetime'] = pd.to_datetime(df_ground_truth.datetime)
df_ground_truth['date'] = pd.to_datetime(df_ground_truth.datetime.dt.date, format='%Y-%m-%d')
df_yolo_predictions['datetime'] = pd.to_datetime(df_yolo_predictions['datetime'])
df_yolo_predictions['date'] = pd.to_datetime(df_yolo_predictions.datetime.dt.date, format='%Y-%m-%d')

df_ground_truth = df_ground_truth[df_ground_truth.date.isin(df_yolo_predictions.date.unique())]

print('shape of loaded data', df_ground_truth.shape, df_yolo_predictions.shape)
print('Ground truth data:')
print(df_ground_truth.groupby(['date', 'camera_id']).datetime.count())
print('YOLO predictions data:')
print(df_yolo_predictions.groupby(['date', 'camera_id']).datetime.count())

# crop bounding boxes from right side of camera 2 field of view
# df_yolo_predictions.drop(index=df_yolo_predictions[(df_yolo_predictions.camera_id == 2) & (df_yolo_predictions.x > 1800)].index, inplace=True)
# df_ground_truth.drop(index=df_ground_truth[(df_ground_truth.camera_id == 2) & (df_ground_truth.x > 1800)].index, inplace=True)

df_ground_truth.set_index('filename', inplace=True)
df_yolo_predictions.set_index('filename', inplace=True)

df_ground_truth.shape, df_yolo_predictions.shape

shape of loaded data (11492, 11) (127468, 11)
Ground truth data:
date        camera_id
2023-07-07  2            6500
2023-07-08  1            4992
Name: datetime, dtype: int64
YOLO predictions data:
date        camera_id
2023-07-07  2            105364
2023-07-08  1             22104
Name: datetime, dtype: int64


((11492, 10), (127468, 10))

In [5]:
# transform coords to xmin, ymin, xmax, ymax
df_ground_truth['xmin'] = (df_ground_truth['x'] - df_ground_truth['w']/2).astype(int)
df_ground_truth['ymin'] = (df_ground_truth['y'] - df_ground_truth['h']/2).astype(int)
df_ground_truth['xmax'] = (df_ground_truth['x'] + df_ground_truth['w']/2).astype(int)
df_ground_truth['ymax'] = (df_ground_truth['y'] + df_ground_truth['h']/2).astype(int)
df_yolo_predictions['xmin'] = (df_yolo_predictions['x'] - df_yolo_predictions['w']/2).astype(int)
df_yolo_predictions['ymin'] = (df_yolo_predictions['y'] - df_yolo_predictions['h']/2).astype(int)
df_yolo_predictions['xmax'] = (df_yolo_predictions['x'] + df_yolo_predictions['w']/2).astype(int)
df_yolo_predictions['ymax'] = (df_yolo_predictions['y'] + df_yolo_predictions['h']/2).astype(int)

In [6]:
# filter out predicitons in the bank on camera 1; use a line to define the bank in the camera 1 field of view
point1_cam01_bank = (489, 591)
point2_cam01_bank = (1499, 875)
slope_cam01_bank = (point2_cam01_bank[1] - point1_cam01_bank[1]) / (point2_cam01_bank[0] - point1_cam01_bank[0])
intercept_cam01_bank = point1_cam01_bank[1] - slope_cam01_bank * point1_cam01_bank[0]

def under_the_bank_apply(row):
    if row.camera_id == 1 and row.x >= point1_cam01_bank[0] and row.x <= point2_cam01_bank[0]:
        if row.y > (slope_cam01_bank * row.x + intercept_cam01_bank):
            return True        
    return False
        
df_yolo_predictions['under_the_bank'] = df_yolo_predictions.apply(under_the_bank_apply, axis=1)
print('How many prediction were under the bank?')
print(df_yolo_predictions['under_the_bank'].value_counts())
df_yolo_predictions = df_yolo_predictions[df_yolo_predictions.under_the_bank == False].copy()
print('YOLO predictions filtered data:')
print(df_yolo_predictions.groupby(['date', 'camera_id']).datetime.count())

How many prediction were under the bank?
under_the_bank
False    120382
True       7086
Name: count, dtype: int64
YOLO predictions filtered data:
date        camera_id
2023-07-07  2            105364
2023-07-08  1             15018
Name: datetime, dtype: int64


In [7]:
df_ground_truth_cam1 = df_ground_truth[df_ground_truth['camera_id'] == 1]
df_prediction_cam_1 = df_yolo_predictions[df_yolo_predictions['camera_id'] == 1]
df_ground_truth_cam2 = df_ground_truth[df_ground_truth['camera_id'] == 2]
df_prediction_cam_2 = df_yolo_predictions[df_yolo_predictions['camera_id'] == 2]
df_ground_truth_cam1.shape, df_prediction_cam_1.shape, df_ground_truth_cam2.shape, df_prediction_cam_2.shape

((4992, 14), (15018, 15), (6500, 14), (105364, 15))

# Evaluation of precision and recall for detected frame_ids

In [8]:
df_evaluation, total_evaluation_dict = model_evaluation_functions.evaluate_model_frame_ids_score(df_ground_truth, df_yolo_predictions)
total_evaluation_dict

  0%|          | 0/145 [00:00<?, ?it/s]

100%|██████████| 145/145 [00:17<00:00,  8.08it/s]


{'f1': 0.18875570387522986,
 'recall': 0.9824530308401276,
 'precision': 0.10440760971934451,
 'iou': 0.7513576454399039}

In [9]:
df_evaluation.to_csv('data_evaluation_dual_model.csv', sep=';')

In [10]:
model_evaluation_functions.evaluate_mAP(df_ground_truth, df_yolo_predictions)

  2%|▏         | 2/84 [00:00<00:06, 13.41it/s]

100%|██████████| 84/84 [00:22<00:00,  3.67it/s]


{'mAP_0.5_11points': 0.24341293,
 'mAP_0.5_all_points': 0.19430423,
 'mAP_0.5_0.95_101points': 0.10441625}

In [11]:
df_evaluation, total_evaluation_dict = model_evaluation_functions.evaluate_model_frame_ids_score(df_ground_truth_cam1, df_prediction_cam_1)
total_evaluation_dict

  0%|          | 0/51 [00:00<?, ?it/s]

100%|██████████| 51/51 [00:06<00:00,  7.41it/s]


{'f1': 0.5085289556708198,
 'recall': 0.9775904605263158,
 'precision': 0.34364385343643855,
 'iou': 0.7691296548989031}

In [12]:
model_evaluation_functions.evaluate_mAP(df_ground_truth_cam1, df_prediction_cam_1)

  0%|          | 0/33 [00:00<?, ?it/s]

100%|██████████| 33/33 [00:05<00:00,  5.67it/s]


{'mAP_0.5_11points': 0.34617582,
 'mAP_0.5_all_points': 0.36040094,
 'mAP_0.5_0.95_101points': 0.21119758}

In [13]:
df_evaluation, total_evaluation_dict = model_evaluation_functions.evaluate_model_frame_ids_score(df_ground_truth_cam2, df_prediction_cam_2)
total_evaluation_dict

  5%|▌         | 5/94 [00:00<00:11,  7.53it/s]

100%|██████████| 94/94 [00:10<00:00,  9.19it/s]


{'f1': 0.12820590707046162,
 'recall': 0.9861370716510903,
 'precision': 0.06855960928278267,
 'iou': 0.7380096901441305}

In [15]:
model_evaluation_functions.evaluate_mAP(df_ground_truth_cam2, df_prediction_cam_2)

  0%|          | 0/51 [00:00<?, ?it/s]

100%|██████████| 51/51 [00:17<00:00,  2.84it/s]


{'mAP_0.5_11points': 0.17983977,
 'mAP_0.5_all_points': 0.1613526,
 'mAP_0.5_0.95_101points': 0.08039116}

## Use validation part of data and only a filter images with known ground truth

In [16]:
df_ground_truth_filtered = df_ground_truth[(df_ground_truth.date == '2023-07-07') | (df_ground_truth.date == '2023-07-08')].reset_index().set_index(['filename', 'frame_id'])
df_yolo_predictions_filtered = df_yolo_predictions.reset_index().set_index(['filename', 'frame_id'])
df_yolo_predictions_filtered = df_yolo_predictions_filtered.loc[df_ground_truth_filtered.index.intersection(df_yolo_predictions_filtered.index)]
df_ground_truth_filtered.shape, df_yolo_predictions_filtered.shape

((11492, 13), (13734, 14))

In [17]:
df_ground_truth_filtered = df_ground_truth_filtered.reset_index().set_index('filename')
df_yolo_predictions_filtered = df_yolo_predictions_filtered.reset_index().set_index('filename')
df_ground_truth_filtered_cam1 = df_ground_truth_filtered[df_ground_truth_filtered['camera_id'] == 1]
df_prediction_filtered_cam_1 = df_yolo_predictions_filtered[df_yolo_predictions_filtered['camera_id'] == 1]
df_ground_truth_filtered_cam2 = df_ground_truth_filtered[df_ground_truth_filtered['camera_id'] == 2]
df_prediction_filtered_cam_2 = df_yolo_predictions_filtered[df_yolo_predictions_filtered['camera_id'] == 2]
df_ground_truth_filtered_cam1.shape, df_prediction_filtered_cam_1.shape, df_ground_truth_filtered_cam2.shape, df_prediction_filtered_cam_2.shape

((4992, 14), (5681, 15), (6500, 14), (8053, 15))

In [18]:
df_evaluation, total_evaluation_dict = model_evaluation_functions.evaluate_model_frame_ids_score(df_ground_truth_filtered, df_yolo_predictions_filtered)
total_evaluation_dict

  5%|▍         | 4/84 [00:00<00:12,  6.47it/s]

100%|██████████| 84/84 [00:14<00:00,  5.97it/s]


{'f1': 0.9911488600804649,
 'recall': 0.9824530308401276,
 'precision': 1.0,
 'iou': 0.751357645439904}

In [19]:
model_evaluation_functions.evaluate_mAP(df_ground_truth_filtered, df_yolo_predictions_filtered)

  0%|          | 0/84 [00:00<?, ?it/s]

100%|██████████| 84/84 [00:08<00:00,  9.56it/s]


{'mAP_0.5_11points': 0.87064517,
 'mAP_0.5_all_points': 0.88445246,
 'mAP_0.5_0.95_101points': 0.4525165}

In [20]:
df_evaluation, total_evaluation_dict = model_evaluation_functions.evaluate_model_frame_ids_score(df_ground_truth_filtered_cam1, df_prediction_filtered_cam_1)
total_evaluation_dict

  0%|          | 0/33 [00:00<?, ?it/s]

100%|██████████| 33/33 [00:06<00:00,  5.50it/s]


{'f1': 0.9886682607339641,
 'recall': 0.9775904605263158,
 'precision': 1.0,
 'iou': 0.769129654898903}

In [21]:
model_evaluation_functions.evaluate_mAP(df_ground_truth_filtered_cam1, df_prediction_filtered_cam_1)

  0%|          | 0/33 [00:00<?, ?it/s]

100%|██████████| 33/33 [00:03<00:00,  9.06it/s]


{'mAP_0.5_11points': 0.80302376,
 'mAP_0.5_all_points': 0.87615025,
 'mAP_0.5_0.95_101points': 0.50071406}

In [22]:
df_evaluation, total_evaluation_dict = model_evaluation_functions.evaluate_model_frame_ids_score(df_ground_truth_filtered_cam2, df_prediction_filtered_cam_2)
total_evaluation_dict

  4%|▍         | 2/51 [00:00<00:06,  7.29it/s]

100%|██████████| 51/51 [00:08<00:00,  6.22it/s]


{'f1': 0.9930201552819387,
 'recall': 0.9861370716510903,
 'precision': 1.0,
 'iou': 0.7380096901441308}

In [23]:
model_evaluation_functions.evaluate_mAP(df_ground_truth_filtered_cam2, df_prediction_filtered_cam_2)

  4%|▍         | 2/51 [00:00<00:06,  7.54it/s]

100%|██████████| 51/51 [00:04<00:00, 10.25it/s]


{'mAP_0.5_11points': 0.8917491,
 'mAP_0.5_all_points': 0.91879463,
 'mAP_0.5_0.95_101points': 0.43354067}